In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from keybert import KeyBERT
import yake
import string
import numpy as np

In [2]:
sds_data = pd.read_csv('../data/sds_ds_text.csv')
sds_non_data = pd.read_csv('../data/sds_nds_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Generating keywords for Data Science Episodes

In [3]:
sds_ds_processed = pd.DataFrame(sds_data.groupby(['episode_number', 'episode_name', 'length_episode', 'class', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

<IPython.core.display.Javascript object>

In [4]:
sds_ds_processed

,episode_number,episode_name,length_episode,class,guest_name,host_episode,episode_year,episode_split_text
0,1,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Database,Ruben Kogel,Kirill Eremenko,2016,This is episode number one with ex-chemical en...
1,2,"Machine Learning, Recommender Systems and the...",51,Machine Learning,Hadelin de Ponteves,Kirill Eremenko,2016,This is session number two with machine learni...
2,3,"Defining the Data Problem, Data Science in Ma...",53,Machine Learning,Dr. Wilson Pok,Kirill Eremenko,2016,This is episode number three with Nanophysics ...
3,4,"Data and Strategy, three Pillars of Research ...",60,Data Science,Brendan Hogan,Kirill Eremenko,2016,This is episode four with business strategy ex...
4,5,"Computer Forensics, Fraud Analytics and knowi...",63,Data Science,Dmitry Korneev,Kirill Eremenko,2016,This is episode number five with forensics inv...
...,...,...,...,...,...,...,...,...
446,679,"The A.I. and Machine Learning Landscape, with...",94,Artificial Intelligence,George Mathew,Jon Krohn,2023,This is episode number 679 with George Matthew...
447,680,Automating Industrial Machines with Data Scie...,30,Data Science,Allegra Alessi,Jon Krohn,2023,This is episode number 680 with Allegra Alessi...
448,681,XGBoost,72,Machine Learning,Matt Harrison,Jon Krohn,2023,"This is episode number 681 with Matt Harrison,..."
449,682,"Business Intelligence Tools, with Mico Yuk",28,Data Science,Mico Yuk,Jon Krohn,2023,"This is episode number 682 with Mico Yuk, host..."


In [ ]:
# Remove episode 202 and 546 with no text in episode_split_text
#sds_processed = 
#sds_ds_processed[sds_ds_processed['episode_split_text'] == 'nan']#.reset_index(drop = True)

In [5]:
# Function to clean the text 
def processed_text(text):  
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove digits from the text
    text = ''.join(char for char in text if not char.isdigit())
    return text

In [6]:
sds_ds_processed['episode_split_text'] = sds_ds_processed['episode_split_text'].apply(processed_text)

In [11]:
sds_ds_processed['episode_split_text'].loc[400]

'this is episode number  with luke barousse of the eponymous data analytics youtube channel this episode is brought to you by iterative your mission control center for machine learningwelcome to the super data science podcast the most listenedto podcast in the data science industry each week we bring you inspiring people and ideas to help you build a successful career in data science im your host jon krohn thanks for joining me today and now lets make the complex simplewelcome back to the super data science podcast weve got the tremendous luke barousse on the show today luke is a fulltime youtuber creating highly educational but nevertheless often hilarious videos on data analytics for his enormous audience of over a quarter million subscribers previously luke worked as a lead data analyst and data engineer at basf the global chemical manufacturing giant before kicking off his career in industry he worked for seven years in the us navy on nuclear powered submarines and he holds a degre

In [12]:
sds_ds_processed['episode_keywords'] = None

In [13]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_ds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_ds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_ds_processed['episode_split_text'])

In [ ]:
sds_ds_processed

In [ ]:
def tuple_extractor(row):
    words = [t[0].replace(" ", "_") for t in row]
    return " ".join(words)

In [ ]:
sds_ds_processed['keywords'] = sds_ds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_ds_processed

In [ ]:
sds_ds_processed.to_csv('../data/sds_ds_keywords.csv', index = False)

## Generating keywords for non-Data Science Episodes

In [ ]:
sds_nds_processed = pd.DataFrame(sds_non_data.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_year'])['episode_split_text'].agg(lambda x: ' '.join(str(i) for i in x))).reset_index()

In [ ]:
sds_nds_processed['episode_split_text'] = sds_nds_processed['episode_split_text'].apply(processed_text)

In [ ]:
sds_nds_processed['episode_keywords'] = None

In [ ]:
# Function to generate 100 keywords per episode using Yake library to store keywords in one column as list
def yake_keyword_extractor(doc):
    global sds_nds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_nds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_nds_processed['episode_split_text'])

In [ ]:
sds_nds_processed['keywords'] = sds_nds_processed['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_nds_processed

In [ ]:
sds_nds_processed.to_csv('../data/sds_nds_keywords.csv', index = False)

### Count Vectorizer and Class-Based Tfidf

In [ ]:
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import normalize

In [ ]:
sds_ds_keywords = pd.read_csv('../data/sds_ds_keywords.csv')

keywords_per_class = sds_ds_keywords.groupby('context_episode')['keywords'].apply(' '.join).reset_index()

# Rename the columns to match the original code
keywords_per_class = keywords_per_class.rename(columns={'context_episode': 'Class', 'keywords': 'Document'})

In [ ]:
keywords_per_class

In [ ]:
class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """Learn the idf vector (global term weights) """
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        idf = np.log(n_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=np.float64)
        return self

    def transform(self, X: sp.csr_matrix) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF """
        X = X * self._idf_diag
        X = normalize(X, axis=1, norm='l1', copy=False)
        return X

In [ ]:
# Create c-TF-IDF
count = CountVectorizer().fit_transform(keywords_per_class.Document)
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(keywords_per_class)).toarray()

In [ ]:
# Create bag of words using count vectorizer
count_vectorizer = CountVectorizer().fit(keywords_per_class.Document)
count = count_vectorizer.transform(keywords_per_class.Document)
words = count_vectorizer.get_feature_names_out()

In [ ]:
count_vectorizer.vocabulary_

In [ ]:
len(count_vectorizer.vocabulary_)

In [ ]:
count.shape

In [ ]:
word_counts = pd.DataFrame({
    'words': count_vectorizer.get_feature_names_out(),
    'frequency': np.array(count.sum(axis = 0)).flatten()
})
word_counts.sort_values('frequency', ascending = False)

In [ ]:
word_counts[word_counts['frequency'] < 5]

In [ ]:
# class based tfidf
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(keywords_per_class)).toarray()

In [ ]:
# find unique words in each class
unique_words_per_class = []
for i, class_docs in enumerate(keywords_per_class.Document):
    tfidf_scores = ctfidf[i]
    unique_word_indices = np.argsort(tfidf_scores)[-20:][::-1]  # index
    unique_words = [words[idx] for idx in unique_word_indices]  # feature name
    unique_words_per_class.append(unique_words)

class_unique_20 = pd.DataFrame({'Class': keywords_per_class.Class, 'unique_Words': unique_words_per_class})


In [ ]:
class_unique_20

In [ ]:
class_unique_20['unique_Words'].loc[0]

In [ ]:
class_unique_20['Class'].unique()

In [ ]:
class_unique_20.to_csv('../data/keywords_u20.csv')

### Generate word cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:

all_keywords = ' '.join(keywords_per_class.Document)

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_keywords)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
all_keywords = ' '.join(map(str, class_unique_20.unique_Words))

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_keywords)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Function to generate 100 keywords per episode using Yake library if keywords need to be separate columns

def yake_keyword_extractor(doc):
    global sds_processed

    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n=max_ngram_size, dedupLim=deduplication_threshold, windowsSize=windowSize, top=numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)

        # Create separate columns for each keyword
        for j, keyword in enumerate(keywords):
            column_name = f'episode_keyword{j+1}'
            sds_processed.at[i, column_name] = keyword[0]  # Store the keyword value in the respective column

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
# https://maartengr.github.io/BERTopic/index.html